In [17]:
import os, subprocess

In [18]:
def runNSave(cmd, path):
    result = subprocess.run(cmd, stdout=subprocess.PIPE)
    with open(path, 'w') as fh:
        fh.write(result.stdout.decode("utf-8"))

In [19]:
path = "/Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/other_systems/gmark-dominik/50/"
for file in os.listdir(path):    
    if not file.endswith(".sparql"):
        continue
    
    file = file[:file.index(".")] + ".n3"
    or_file = os.path.join(path, "n3", file)
    norm_file = os.path.join(path, "n3", "normalized", file)
    
    print(file)
    
    # ground
    runNSave(["eye", "--quiet", or_file, "--no-qvars", "--nope", "--pass-all"], norm_file)
    
    # normalize
    runNSave(["eye", "--quiet", norm_file, "aux_list.n3", "--query", "list-predicate.n3", "--quantify", "http://www.w3.org/2000/10/swap/var#", "--nope"], norm_file)

query-24.n3
query-47.n3
query-36.n3
query-28.n3
query-12.n3
query-1.n3
query-49.n3
query-38.n3
query-45.n3
query-10.n3
query-7.n3
query-14.n3
query-22.n3
query-16.n3
query-5.n3
query-32.n3
query-20.n3
query-43.n3
query-0.n3
query-13.n3
query-46.n3
query-25.n3
query-29.n3
query-37.n3
query-11.n3
query-2.n3
query-48.n3
query-35.n3
query-27.n3
query-44.n3
query-39.n3
query-23.n3
query-40.n3
query-19.n3
query-6.n3
query-21.n3
query-4.n3
query-17.n3
query-8.n3


In [20]:
path = "/Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/other_systems/gmark-dominik/50/"
files = list(os.listdir(os.path.join(path, "n3", "normalized")))
files.sort()
for file in files:
    if not file.startswith("query") and not file.endswith(".n3"):
        continue
    
    norm_file = os.path.join(path, "n3", "normalized", file)
    data_path = os.path.join(path, "data.n3")
    dir_res_file = os.path.join(path, "n3", "results", "direct", file)
    bwd_res_file = os.path.join(path, "n3", "results", "bwd", file)
    fwd_res_file = os.path.join(path, "n3", "results", "fwd", file)
    tmp_file = "tmp.n3"
    
    print(file, dir_res_file)
    
    # - direct 
    runNSave(["eye", data_path, "property-paths-direct.n3", "--query", norm_file, "--nope"], dir_res_file)
    
    # - backward
    runNSave(["eye", norm_file, "rule-creation.n3", "--query", "rule-creation-backwards.n3", "--nope"], tmp_file)
    runNSave(["eye", data_path, tmp_file, "--query", norm_file, "--nope"], bwd_res_file)
        
    # - forward
    runNSave(["eye", norm_file, "rule-creation.n3", "--query", "rule-creation.n3", "--nope"], tmp_file)
    runNSave(["eye", data_path, tmp_file, "--query", norm_file, "--nope"], fwd_res_file)

query-0.n3 /Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/other_systems/gmark-dominik/50/n3/results/direct/query-0.n3


eye --quiet /Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/other_systems/gmark-dominik/50/data.n3 property-paths-direct.n3 --query /Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/other_systems/gmark-dominik/50/n3/normalized/query-0.n3 --nope
EYE v10.24.10 (2024-10-02)
SWI-Prolog version 9.2.7
starting 28 [msec cputime] 31 [msec walltime]
GET file:///Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/other_systems/gmark-dominik/50/data.n3 SC=55
GET file:///Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/SPIN-to-N3/property-paths/property-paths-direct.n3 SC=14
GET file:///Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/other_systems/gmark-dominik/50/n3/normalized/query-0.n3 SC=1
networking 7 [msec cputime] 8 [msec walltime]
reasoning 201 [msec cputime] 205 [msec walltime]
2025-05-20T13:47:59.556Z in=70 out=5 ent=5 step=82528 brake=1 inf=3223800 sec=0.236 inf/sec=13660169

eye --quiet /Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/other_systems/gmark-dominik/50/n3/normalized/query-0.n3 rule-creation.n3 --query rule-creation-backwards

query-1.n3 /Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/other_systems/gmark-dominik/50/n3/results/direct/query-1.n3


eye --quiet /Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/other_systems/gmark-dominik/50/n3/normalized/query-1.n3 rule-creation.n3 --query rule-creation-backwards.n3 --nope
EYE v10.24.10 (2024-10-02)
SWI-Prolog version 9.2.7
starting 25 [msec cputime] 28 [msec walltime]
GET file:///Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/other_systems/gmark-dominik/50/n3/normalized/query-1.n3 SC=1
GET file:///Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/SPIN-to-N3/property-paths/rule-creation.n3 SC=11
GET file:///Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/SPIN-to-N3/property-paths/rule-creation-backwards.n3 SC=1
networking 8 [msec cputime] 8 [msec walltime]
reasoning 22 [msec cputime] 24 [msec walltime]
2025-05-20T13:48:00.043Z in=13 out=18 ent=38 step=151 brake=5 inf=419239 sec=0.055 inf/sec=7622527

eye --quiet /Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/other_systems/gmark-dominik/50/data.n3 tmp.n3 --query /Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/other_systems/gmark-dominik/50/n3/normalized/query-1.n3 --nope
EYE v1

query-10.n3 /Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/other_systems/gmark-dominik/50/n3/results/direct/query-10.n3


...

reasoning 9557 [msec cputime] 9656 [msec walltime]
2025-05-20T13:50:14.809Z in=70 out=297 ent=297 step=933900 brake=1 inf=38971904 sec=9.588 inf/sec=4064654

eye --quiet /Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/other_systems/gmark-dominik/50/n3/normalized/query-6.n3 rule-creation.n3 --query rule-creation-backwards.n3 --nope
EYE v10.24.10 (2024-10-02)
SWI-Prolog version 9.2.7
starting 23 [msec cputime] 26 [msec walltime]
GET file:///Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/other_systems/gmark-dominik/50/n3/normalized/query-6.n3 SC=1
GET file:///Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/SPIN-to-N3/property-paths/rule-creation.n3 SC=11
GET file:///Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/SPIN-to-N3/property-paths/rule-creation-backwards.n3 SC=1
networking 7 [msec cputime] 8 [msec walltime]
reasoning 15 [msec cputime] 16 [msec walltime]
2025-05-20T13:50:14.869Z in=13 out=11 ent=23 step=86 brake=4 inf=324687 sec=0.045 inf/sec=7215267

eye --quiet /Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/other_s

query-7.n3 /Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/other_systems/gmark-dominik/50/n3/results/direct/query-7.n3


eye --quiet /Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/other_systems/gmark-dominik/50/data.n3 property-paths-direct.n3 --query /Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/other_systems/gmark-dominik/50/n3/normalized/query-7.n3 --nope
EYE v10.24.10 (2024-10-02)
SWI-Prolog version 9.2.7
starting 24 [msec cputime] 27 [msec walltime]
GET file:///Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/other_systems/gmark-dominik/50/data.n3 SC=55
GET file:///Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/SPIN-to-N3/property-paths/property-paths-direct.n3 SC=14
GET file:///Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/other_systems/gmark-dominik/50/n3/normalized/query-7.n3 SC=1
networking 7 [msec cputime] 9 [msec walltime]
reasoning 664 [msec cputime] 676 [msec walltime]
2025-05-20T13:50:20.136Z in=70 out=598 ent=598 step=28416 brake=1 inf=3798705 sec=0.695 inf/sec=5465763

eye --quiet /Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/other_systems/gmark-dominik/50/n3/normalized/query-7.n3 rule-creation.n3 --query rule-creation-backwa

query-8.n3 /Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/other_systems/gmark-dominik/50/n3/results/direct/query-8.n3


reasoning 2037 [msec cputime] 2086 [msec walltime]
2025-05-20T13:50:23.082Z in=70 out=857 ent=857 step=85578 brake=1 inf=4636875 sec=2.069 inf/sec=2241119

eye --quiet /Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/other_systems/gmark-dominik/50/n3/normalized/query-8.n3 rule-creation.n3 --query rule-creation-backwards.n3 --nope
EYE v10.24.10 (2024-10-02)
SWI-Prolog version 9.2.7
starting 24 [msec cputime] 27 [msec walltime]
GET file:///Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/other_systems/gmark-dominik/50/n3/normalized/query-8.n3 SC=1
GET file:///Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/SPIN-to-N3/property-paths/rule-creation.n3 SC=11
GET file:///Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/SPIN-to-N3/property-paths/rule-creation-backwards.n3 SC=1
networking 7 [msec cputime] 8 [msec walltime]
reasoning 24 [msec cputime] 24 [msec walltime]
2025-05-20T13:50:23.154Z in=13 out=17 ent=36 step=147 brake=5 inf=425022 sec=0.055 inf/sec=7727673

eye --quiet /Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/other_sy